# Clustering Zooniverse Marks to count Iguanas
Count all the iguanas in the images by clustering the marks from the zooniverse volunteers.
This does not compare the results to the gold standard and requires only a single file, the flattened zooniverse data export.

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("./")
sys.path.append("./zooniverse")

## Intro
### Retrieve a Classification report from Zooniverse
This notebook is used to cluster the marks from the zooniverse volunteers to count the iguanas in the images saved in results/<phase_tag>/flat_dataset_filtered_Iguanas 3rd launch.csv

Used Methods are:

### DBSCAN 
It does not require the number of clusters to be specified. It is used here because, but has min_samples and eps as hyperparameters which need to be found. [Link](https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html)
For finding eps and min_sample a simple **grid search** is used.
Additionally, DBSCAN not assume a specific shape for the clusters (K-means assumes clusters are gaussian in shape) even though we should assume that points around an iguana is gaussian shaped.

### HDBSCAN
It is an extension of DBSCAN which is more robust to hyperparameter settings as it finds epsilon and min_samples automatically. [Link](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html)

## Load the data

In [2]:
from pathlib import Path

import pandas as pd
from zooniverse.analysis import get_annotation_count_stats
from zooniverse.utils.filters import filter_df_user_threshold



# Phase Selection
# phase_tag = "Iguanas 1st launch"
phase_tag = "Iguanas 2nd launch"
# phase_tag = "Iguanas 3rd launch"


debug = False # debugging with a smaller dataset
plot_diagrams = False # plot the diagrams to disk for the clustering methods
show_plots = False # show the plots in the notebook

user_threshold = None # None or a number, filter records which have less than these user interactions.
use_gold_standard_subset = None # Use no filtering


## Input Path of all the data
input_path =Path("/Users/christian/data/zooniverse")
# input_path = Path("results/")
# Location for the analysis Results
output_path = Path(input_path.joinpath(f"2024_04_12_analysis").joinpath(phase_tag))
output_path.mkdir(exist_ok=True, parents=True)
reprocess = False # if True, the raw classification data is reprocessed. If False, the data is loaded from disk

# Location for plots
# output_plot_path = output_path.joinpath("plots")
# output_plot_path.mkdir(parents=True, exist_ok=True)
output_plot_path = None # do not plot

## Look into the config
This Config points to all files necessary for the analysis + the result files

In [3]:
from zooniverse.config import get_config

config = get_config(phase_tag=phase_tag, input_path=input_path, output_path=output_path)
config

{'annotations_source': PosixPath('/Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv'),
 'goldstandard_data': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/expert-GS-2ndphase.csv'),
 'gold_standard_image_subset': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/2-T2-GS-results-5th-0s.csv'),
 'image_source': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/2nd launch_without_prefix'),
 'yes_no_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_12_analysis/Iguanas 2nd launch/yes_no_dataset_Iguanas 2nd launch.csv'),
 'flat_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_12_analysis/Iguanas 2nd launch/flat_dataset_Iguanas 2nd launch.csv'),
 'flat_panoptes_points': PosixPath('/Users/christian/data/zooniverse/2024_04_12_analysis/Iguanas 2nd launch/flat_panoptes_points_Iguanas 2nd launch.csv'),
 'panoptes_question': Pos

In [4]:
config["flat_panoptes_points"]

PosixPath('/Users/christian/data/zooniverse/2024_04_12_analysis/Iguanas 2nd launch/flat_panoptes_points_Iguanas 2nd launch.csv')

## Look at the data


In [5]:

df_flat_panoptes_points = pd.read_csv(config["flat_panoptes_points"])
df_flat_panoptes_points["phase"] = phase_tag

df_flat_panoptes_points

,classification_id,user_name,user_id,workflow_id,task,created_at,subject_id,image_name,x,y,phase
0,397130915,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-02-15 19:21:11 UTC,72332826,EGI01-1-2_83.jpg,297,102,Iguanas 2nd launch
1,397130915,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-02-15 19:21:11 UTC,72332826,EGI01-1-2_83.jpg,297,102,Iguanas 2nd launch
2,401114209,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-03-08 00:53:47 UTC,72332844,EGI01-2-1_26.jpg,1384,1027,Iguanas 2nd launch
3,401114209,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-03-08 00:53:47 UTC,72332844,EGI01-2-1_26.jpg,1384,1027,Iguanas 2nd launch
4,397739195,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-02-18 20:16:43 UTC,72332945,EGI02-1_23.jpg,679,338,Iguanas 2nd launch
...,...,...,...,...,...,...,...,...,...,...,...
177227,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,97,117,Iguanas 2nd launch
177228,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,113,186,Iguanas 2nd launch
177229,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,87,167,Iguanas 2nd launch
177230,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,443,124,Iguanas 2nd launch


### Filter User if necessary and Marks


In [6]:
# Use the panoptes Extraction instead of the custom extraction
df_merged_dataset = df_flat_panoptes_points

### Are there anonymous users in the data?
There should be some because anonymous users are kept.

In [7]:
df_merged_dataset[df_merged_dataset.user_id.isnull().values]

,classification_id,user_name,user_id,workflow_id,task,created_at,subject_id,image_name,x,y,phase
156432,394601142,22f2858fc7b6a25d0d8910cb081899f0,NaN,20600,T2,2022-02-04 18:24:39 UTC,72332804,EGI01-1-2_24.jpg,588,209,Iguanas 2nd launch
156433,394601142,22f2858fc7b6a25d0d8910cb081899f0,NaN,20600,T2,2022-02-04 18:24:39 UTC,72332804,EGI01-1-2_24.jpg,588,209,Iguanas 2nd launch
156434,396273092,ac28275d5c75246a09823bc8a9b55c33,NaN,20600,T2,2022-02-11 11:18:06 UTC,72332808,EGI01-1-2_38.jpg,1471,-659,Iguanas 2nd launch
156435,396273092,ac28275d5c75246a09823bc8a9b55c33,NaN,20600,T2,2022-02-11 11:18:06 UTC,72332808,EGI01-1-2_38.jpg,1379,-275,Iguanas 2nd launch
156436,396273092,ac28275d5c75246a09823bc8a9b55c33,NaN,20600,T2,2022-02-11 11:18:06 UTC,72332808,EGI01-1-2_38.jpg,301,347,Iguanas 2nd launch
...,...,...,...,...,...,...,...,...,...,...,...
177227,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,97,117,Iguanas 2nd launch
177228,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,113,186,Iguanas 2nd launch
177229,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,87,167,Iguanas 2nd launch
177230,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,443,124,Iguanas 2nd launch


In [8]:
# Amount of images
df_merged_dataset["subject_id"].nunique()

3512

In [9]:
## After filtering there
df_merged_dataset

,classification_id,user_name,user_id,workflow_id,task,created_at,subject_id,image_name,x,y,phase
0,397130915,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-02-15 19:21:11 UTC,72332826,EGI01-1-2_83.jpg,297,102,Iguanas 2nd launch
1,397130915,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-02-15 19:21:11 UTC,72332826,EGI01-1-2_83.jpg,297,102,Iguanas 2nd launch
2,401114209,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-03-08 00:53:47 UTC,72332844,EGI01-2-1_26.jpg,1384,1027,Iguanas 2nd launch
3,401114209,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-03-08 00:53:47 UTC,72332844,EGI01-2-1_26.jpg,1384,1027,Iguanas 2nd launch
4,397739195,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-02-18 20:16:43 UTC,72332945,EGI02-1_23.jpg,679,338,Iguanas 2nd launch
...,...,...,...,...,...,...,...,...,...,...,...
177227,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,97,117,Iguanas 2nd launch
177228,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,113,186,Iguanas 2nd launch
177229,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,87,167,Iguanas 2nd launch
177230,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,443,124,Iguanas 2nd launch


In [10]:
# how many marks per user
df_merged_dataset[["user_id", "user_name", "x"]].groupby("user_name").count()

,user_id,x
user_name,,
001a4ac0175c2415dcffc072cd58094b,0,4
001a7e5e989807349d0dc5e90eaebf83,126,126
00aeae18e483605548b043dfd15ed411,0,2
01745370acd58674cf1bfca6a9fed8cf,60,60
017c5ec6eaabe1884b9808cfb20ca56c,22,22
...,...,...
fed3e9ec64e72819477c65342b22d19f,1414,1414
ff1d0a02e22b42e6a94f1c8eb54846c3,0,8
ffac078ce61b035a5a0af18d4a9f53f8,18,18


## Clustering

### Basic Statics like mean, median, mode

In [11]:
from zooniverse.analysis import get_mark_overview

basic_stats = []

for image_name, df_image_name in df_merged_dataset.groupby("subject_id"):
    annotations_count = get_mark_overview(df_image_name)

    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=df_image_name.iloc[0]["image_name"], 
                                                         subject_id=df_image_name.iloc[0]["subject_id"])

    ### basic statistics like mean, median, mode
    basic_stats.append(annotations_count_stats)
    

df_basic_stats = pd.DataFrame(basic_stats)    
df_comparison = df_basic_stats


In [12]:
df_basic_stats

,image_name,subject_id,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count
0,EGI01-1-1_19.jpg,72332775,2.0,2.00,2,2,[2],2.0,1,2,[2]
1,EGI01-1-1_54.jpg,72332788,2.0,2.00,2,2,[2],2.0,1,2,[2]
2,EGI01-1-2_110.jpg,72332799,24.0,24.00,24,24,[24],24.0,1,24,[24]
3,EGI01-1-2_121.jpg,72332802,54.0,54.00,54,54,[54],54.0,1,54,[54]
4,EGI01-1-2_122.jpg,72332803,6.0,5.67,6,6,[6],6.0,6,34,"[2, 4, 6, 6, 6, 10]"
...,...,...,...,...,...,...,...,...,...,...,...
3507,ESCG02-2_59.jpg,72373345,2.0,2.00,2,2,[2],2.0,2,4,"[2, 2]"
3508,ESCG02-2_62.jpg,72373347,2.0,2.00,2,2,[2],2.0,4,8,"[2, 2, 2, 2]"
3509,ESCG02-2_63.jpg,72373348,2.0,2.00,2,2,[2],2.0,4,8,"[2, 2, 2, 2]"
3510,ESCG02-2_64.jpg,72373350,14.0,12.80,10,14,"[10, 14]",12.0,5,64,"[10, 10, 14, 14, 16]"


In [13]:
# There might be records with too few annotations if they were not removed before
df_comparison[(df_comparison.sum_annotations_count < 5)].sort_values(by="users", ascending=False)

,image_name,subject_id,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count
3507,ESCG02-2_59.jpg,72373345,2.0,2.0,2,2,[2],2.0,2,4,"[2, 2]"
1089,EPS07-2_156.jpg,72336450,2.0,2.0,2,2,[2],2.0,2,4,"[2, 2]"
1970,ESCH01-3_39.jpg,72337941,2.0,2.0,2,2,[2],2.0,2,4,"[2, 2]"
1126,EPS07-2_221.jpg,72336508,2.0,2.0,2,2,[2],2.0,2,4,"[2, 2]"
2862,FSCE-1.1-2_45.jpg,72340392,2.0,2.0,2,2,[2],2.0,2,4,"[2, 2]"
...,...,...,...,...,...,...,...,...,...,...,...
713,EPS04-1_82.jpg,72335743,4.0,4.0,4,4,[4],4.0,1,4,[4]
717,EPS04-1_91.jpg,72335751,2.0,2.0,2,2,[2],2.0,1,2,[2]
718,EPS04-1_95.jpg,72335755,2.0,2.0,2,2,[2],2.0,1,2,[2]
719,EPS04-1_96.jpg,72335756,2.0,2.0,2,2,[2],2.0,1,2,[2]


In [14]:
# Fill NaN values with 0 because the errors can't be calculated otherwise
df_comparison.fillna(0, inplace=True)


### DBSCAN clustering and take the variant with the best silouette score for each image


In [15]:
## sort by silouette score sorting + dbscan_count
from zooniverse.analysis import compare_dbscan

eps_variants = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
min_samples_variants = [3, 5, 8, 10]
if debug:
    eps_variants = [0.3]
    min_samples_variants = [3]
params = [(eps, min_samples) for eps in eps_variants for min_samples in min_samples_variants]

db_scan_results = {}
db_scan_best_results = []
for image_name, df_image_name in df_merged_dataset.groupby("subject_id"):
    # check for at least 5 marks
    if df_image_name.shape[0] >= 5:
        dbscan_localization = compare_dbscan(
            params=params,
            df_flat=df_image_name,
            output_plot_path=output_plot_path,
            plot=show_plots
        )
    
        db_scan_results[image_name] = pd.DataFrame(dbscan_localization)
        
        # DBSCAN tends to classfy all points as noise if min_samples is too high. Often only a single user marked an iguana.
        # Sillouette Scoring needs a minimum of 2 clusters
        # if there are points in decent radius they will belong to a cluster
        if pd.DataFrame(dbscan_localization).dbscan_count.max() == 1:
            db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
            # If two or more cluster seem to exists take ones with the best Silouette score
        else:  
            # take the best result by silouette score if there are more clusters then 1
            db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values(["dbscan_silouette_score", "dbscan_count"], ascending=[False, False]).iloc[0])
    
df_dbscan_localization = pd.concat([*db_scan_results.values()])
df_scan_best_results = pd.DataFrame(db_scan_best_results)


/var/folders/2k/78nn7s4548986wsjh29rhj9w0000gn/T/ipykernel_15355/2646081209.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_dbscan_localization = pd.concat([*db_scan_results.values()])


In [16]:
df_scan_best_results

,image_name,subject_id,dbscan_count,dbscan_noise,dbscan_silouette_score,dbscan_BIC_score,eps,min_samples
24,EGI01-1-2_110.jpg,72332799,3,10,0.214772,NaN,0.50,3
20,EGI01-1-2_121.jpg,72332802,6,2,0.711606,NaN,0.40,3
20,EGI01-1-2_122.jpg,72332803,2,4,0.873299,NaN,0.40,3
24,EGI01-1-2_24.jpg,72332804,1,6,NaN,NaN,0.50,3
0,EGI01-1-2_38.jpg,72332808,0,8,NaN,NaN,0.01,3
...,...,...,...,...,...,...,...,...
16,ESCG02-2_53.jpg,72373336,10,4,0.824543,NaN,0.30,3
0,ESCG02-2_62.jpg,72373347,0,8,NaN,NaN,0.01,3
0,ESCG02-2_63.jpg,72373348,0,8,NaN,NaN,0.01,3
14,ESCG02-2_64.jpg,72373350,2,14,0.855305,NaN,0.20,8


Here it can be seen why the silouette score is difficult because it is often undefined.

In [17]:
## save the combinations of parameters, which maximized the silouette score.
df_dbscan_localization.to_csv(config["dbscan_hyperparam_grid"])


In [18]:
df_scan_best_results.rename(columns={"dbscan_count": "dbscan_count_sil"}, inplace=True)


df_comparison = df_comparison.merge(df_scan_best_results.drop(["image_name"], axis=1), on='subject_id', how='left')

In [19]:

df_comparison = df_comparison.drop(["dbscan_noise", "dbscan_silouette_score", "eps", "min_samples", "dbscan_BIC_score", "with_noise", "bic_avg"], axis=1, errors="ignore")
df_comparison

,image_name,subject_id,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count,dbscan_count_sil
0,EGI01-1-1_19.jpg,72332775,2.0,2.00,2,2,[2],2.0,1,2,[2],NaN
1,EGI01-1-1_54.jpg,72332788,2.0,2.00,2,2,[2],2.0,1,2,[2],NaN
2,EGI01-1-2_110.jpg,72332799,24.0,24.00,24,24,[24],24.0,1,24,[24],3.0
3,EGI01-1-2_121.jpg,72332802,54.0,54.00,54,54,[54],54.0,1,54,[54],6.0
4,EGI01-1-2_122.jpg,72332803,6.0,5.67,6,6,[6],6.0,6,34,"[2, 4, 6, 6, 6, 10]",2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3507,ESCG02-2_59.jpg,72373345,2.0,2.00,2,2,[2],2.0,2,4,"[2, 2]",NaN
3508,ESCG02-2_62.jpg,72373347,2.0,2.00,2,2,[2],2.0,4,8,"[2, 2, 2, 2]",0.0
3509,ESCG02-2_63.jpg,72373348,2.0,2.00,2,2,[2],2.0,4,8,"[2, 2, 2, 2]",0.0
3510,ESCG02-2_64.jpg,72373350,14.0,12.80,10,14,"[10, 14]",12.0,5,64,"[10, 10, 14, 14, 16]",2.0


### HDBSCAN clustering for each image


In [20]:
from zooniverse.analysis import hdbscan

hdbscan_values = []

eps_variants = [0.0] # 0 is the default
min_cluster_sizes = [5] # 5 is the default


for image_name, df_image_name in df_merged_dataset.groupby("subject_id"):
    annotations_count = get_mark_overview(df_image_name)
    
    image_name=df_image_name.iloc[0]["image_name"]
    subject_id=df_image_name.iloc[0]["subject_id"]
    
    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=image_name, 
                                                         subject_id=subject_id)
    
    # if less than min_cluster_sizes points are available clustering makes no sense
    if df_image_name.shape[0] >= 5: # If num_samples is 5 for the min_cluster_size is 5 there is no point in passing data with less than 5 samples
        params = [(eps, min_cluster_size, max_cluster_size) 
                    for eps in eps_variants
                    for min_cluster_size in min_cluster_sizes
                    for max_cluster_size in [None]
              ]

        df_hdbscan = hdbscan(df_marks=df_image_name[["x", "y"]],
                                     output_path=output_plot_path,
                                     plot=show_plots,
                                     show=show_plots,
                                     image_name=image_name,
                                     subject_id=subject_id,
                                     params=params)
        hdbscan_values.append(df_hdbscan)


df_hdbscan = pd.concat(hdbscan_values)



In [21]:
df_hdbscan.drop(["with_noise"], axis=1, inplace=True)
df_hdbscan

,image_name,subject_id,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,EGI01-1-2_110.jpg,72332799,1,0.0,5,None,6
0,EGI01-1-2_121.jpg,72332802,5,0.0,5,None,0
0,EGI01-1-2_122.jpg,72332803,3,0.0,5,None,0
0,EGI01-1-2_24.jpg,72332804,1,0.0,5,None,4
0,EGI01-1-2_38.jpg,72332808,1,0.0,5,None,2
...,...,...,...,...,...,...,...
0,ESCG02-2_53.jpg,72373336,12,0.0,5,None,12
0,ESCG02-2_62.jpg,72373347,1,0.0,5,None,2
0,ESCG02-2_63.jpg,72373348,1,0.0,5,None,2
0,ESCG02-2_64.jpg,72373350,6,0.0,5,None,4


In [22]:
## Statistics for all images
df_comparison = df_comparison.merge(df_hdbscan.drop(["image_name"], axis=1), on=['subject_id'], how='left')
df_comparison.fillna(0, inplace=True)
df_comparison

,image_name,subject_id,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,EGI01-1-1_19.jpg,72332775,2.0,2.00,2,2,[2],2.0,1,2,[2],0.0,0.0,0.0,0.0,0,0.0
1,EGI01-1-1_54.jpg,72332788,2.0,2.00,2,2,[2],2.0,1,2,[2],0.0,0.0,0.0,0.0,0,0.0
2,EGI01-1-2_110.jpg,72332799,24.0,24.00,24,24,[24],24.0,1,24,[24],3.0,1.0,0.0,5.0,0,6.0
3,EGI01-1-2_121.jpg,72332802,54.0,54.00,54,54,[54],54.0,1,54,[54],6.0,5.0,0.0,5.0,0,0.0
4,EGI01-1-2_122.jpg,72332803,6.0,5.67,6,6,[6],6.0,6,34,"[2, 4, 6, 6, 6, 10]",2.0,3.0,0.0,5.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3507,ESCG02-2_59.jpg,72373345,2.0,2.00,2,2,[2],2.0,2,4,"[2, 2]",0.0,0.0,0.0,0.0,0,0.0
3508,ESCG02-2_62.jpg,72373347,2.0,2.00,2,2,[2],2.0,4,8,"[2, 2, 2, 2]",0.0,1.0,0.0,5.0,0,2.0
3509,ESCG02-2_63.jpg,72373348,2.0,2.00,2,2,[2],2.0,4,8,"[2, 2, 2, 2]",0.0,1.0,0.0,5.0,0,2.0
3510,ESCG02-2_64.jpg,72373350,14.0,12.80,10,14,"[10, 14]",12.0,5,64,"[10, 10, 14, 14, 16]",2.0,6.0,0.0,5.0,0,4.0


In [23]:
df_panoptes_question = pd.read_csv(config["panoptes_question"])
df_panoptes_question

,subject_id,data.no,data.yes
0,72332768,22.0,0.0
1,72332769,21.0,0.0
2,72332770,21.0,0.0
3,72332771,20.0,1.0
4,72332772,21.0,2.0
...,...,...,...
9092,72373345,19.0,2.0
9093,72373347,18.0,4.0
9094,72373348,5.0,16.0
9095,72373350,6.0,15.0


In [24]:
df_comparison.to_csv(config["comparison_dataset"])
print(f"saved {config['comparison_dataset']}")

saved /Users/christian/data/zooniverse/2024_04_12_analysis/Iguanas 2nd launch/Iguanas 2nd launch_method_comparison.csv


## Join the Expert Dataset for comparison
This dataset contains total_counts found by experts. 

In [25]:
df_expert_count = pd.read_csv(config["goldstandard_data"], sep=";")
# check what is in there
df_expert_count[df_expert_count.image_name == "SFM01-1-2_264.jpg"]

,subspecies,island,site_name,subject_group,image_name,subject_id,presence_absence,count_male-lek,count_male-no-lek,count_others,count_partial,count_total,quality,condition,comment


In [26]:
## join the gold standard data to the basic stats
df_comparison_expert = df_expert_count[["site_name", "subject_group", "image_name", "subject_id", "count_total"]].merge(df_comparison.drop(["image_name"], axis=1), on='subject_id', how='left')

columns_to_fill = ['median_count', 'mean_count', 'mode_min_count', 'mode_max_count',
       'mode_count', 'mode_count_avg', 'users', 'sum_annotations_count',
        'dbscan_count_sil', 'HDBSCAN_count']
df_comparison_expert[columns_to_fill] = df_comparison_expert[columns_to_fill].fillna(0)

df_comparison_expert.sort_values(by="median_count", ascending=False)

,site_name,subject_group,image_name,subject_id,count_total,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
179,South Coast G,South Coast G,ESCG02-1_19.jpg,72373250,35,63.0,61.14,58.0,70.0,"[58, 62, 70]",63.333333,14.0,856.0,"[2, 42, 58, 58, 60, 62, 62, 64, 70, 70, 72, 74...",31.0,41.0,0.0,5.0,0.0,40.0
1,Gardner Islet,Gardner Islet,EGI01-1-2_121.jpg,72332802,0,54.0,54.00,54.0,54.0,[54],54.000000,1.0,54.0,[54],6.0,5.0,0.0,5.0,0.0,0.0
296,Punta Ayora,Punta Ayora,FPA04_151.jpg,72339662,24,45.0,37.20,48.0,50.0,"[48, 50]",49.000000,10.0,372.0,"[4, 8, 32, 42, 44, 46, 48, 48, 50, 50]",26.0,25.0,0.0,5.0,0.0,6.0
205,South Coast H,South Coast H,ESCH02-1_112.jpg,72337968,20,32.0,34.00,32.0,32.0,[32],32.000000,8.0,272.0,"[28, 28, 32, 32, 32, 34, 42, 44]",20.0,21.0,0.0,5.0,0.0,13.0
186,South Coast G,South Coast G,ESCG02-2_42.jpg,72373321,20,31.0,31.00,38.0,38.0,[38],38.000000,10.0,310.0,"[8, 24, 26, 28, 30, 32, 38, 38, 42, 44]",5.0,21.0,0.0,5.0,0.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,South Coast H,South Coast H,ESCH02-2_19.jpg,72338235,0,0.0,0.00,0.0,0.0,0,0.000000,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN
223,South Coast H,South Coast H,ESCH02-2_18.jpg,72338234,0,0.0,0.00,0.0,0.0,0,0.000000,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN
219,South Coast H,South Coast H,ESCH02-1_439.jpg,72338173,0,0.0,0.00,0.0,0.0,0,0.000000,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN
216,South Coast H,South Coast H,ESCH02-1_382.jpg,72338147,0,0.0,0.00,0.0,0.0,0,0.000000,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [27]:
df_comparison_expert.to_csv(config["comparison_dataset_expert"])

In [28]:
df_comparison_yes_no = df_comparison_expert.merge(df_panoptes_question, on="subject_id", how="left")
df_comparison_yes_no

,site_name,subject_group,image_name,subject_id,count_total,median_count,mean_count,mode_min_count,mode_max_count,mode_count,...,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points,data.no,data.yes
0,Gardner Islet,Gardner Islet,EGI01-1-2_120.jpg,72332801,0,0.0,0.00,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,23.0,0.0
1,Gardner Islet,Gardner Islet,EGI01-1-2_121.jpg,72332802,0,54.0,54.00,54.0,54.0,[54],...,54.0,[54],6.0,5.0,0.0,5.0,0.0,0.0,19.0,1.0
2,Gardner Islet,Gardner Islet,EGI01-2-2_116.jpg,72332876,0,0.0,0.00,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,21.0,0.0
3,Gardner Islet,Gardner Islet,EGI02-2_114.jpg,72332972,0,0.0,0.00,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,21.0,0.0
4,Gardner Islet,Gardner Islet,EGI02-2_36.jpg,72332999,0,0.0,0.00,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,20.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,South Coast J,South Coast J,FSCJ02-2_243.jpg,72341576,3,2.0,3.20,2.0,2.0,[2],...,32.0,"[2, 2, 2, 2, 2, 2, 4, 4, 4, 8]",2.0,2.0,0.0,5.0,0.0,4.0,6.0,15.0
452,South Coast J,South Coast J,FSCJ02-2_251.jpg,72341585,1,0.0,0.00,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,21.0,0.0
453,South Coast J,South Coast J,FSCJ02-2_269.jpg,72341593,5,10.0,9.45,10.0,10.0,[10],...,104.0,"[4, 6, 6, 8, 8, 10, 10, 10, 10, 12, 20]",5.0,6.0,0.0,5.0,0.0,6.0,3.0,19.0
454,South Coast J,South Coast J,FSCJ02-2_59.jpg,72341609,0,0.0,0.00,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,22.0,0.0


In [29]:
df_comparison_yes_no.to_csv(config["comparison_dataset_yes_no"])
df_comparison_yes_no

,site_name,subject_group,image_name,subject_id,count_total,median_count,mean_count,mode_min_count,mode_max_count,mode_count,...,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points,data.no,data.yes
0,Gardner Islet,Gardner Islet,EGI01-1-2_120.jpg,72332801,0,0.0,0.00,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,23.0,0.0
1,Gardner Islet,Gardner Islet,EGI01-1-2_121.jpg,72332802,0,54.0,54.00,54.0,54.0,[54],...,54.0,[54],6.0,5.0,0.0,5.0,0.0,0.0,19.0,1.0
2,Gardner Islet,Gardner Islet,EGI01-2-2_116.jpg,72332876,0,0.0,0.00,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,21.0,0.0
3,Gardner Islet,Gardner Islet,EGI02-2_114.jpg,72332972,0,0.0,0.00,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,21.0,0.0
4,Gardner Islet,Gardner Islet,EGI02-2_36.jpg,72332999,0,0.0,0.00,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,20.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,South Coast J,South Coast J,FSCJ02-2_243.jpg,72341576,3,2.0,3.20,2.0,2.0,[2],...,32.0,"[2, 2, 2, 2, 2, 2, 4, 4, 4, 8]",2.0,2.0,0.0,5.0,0.0,4.0,6.0,15.0
452,South Coast J,South Coast J,FSCJ02-2_251.jpg,72341585,1,0.0,0.00,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,21.0,0.0
453,South Coast J,South Coast J,FSCJ02-2_269.jpg,72341593,5,10.0,9.45,10.0,10.0,[10],...,104.0,"[4, 6, 6, 8, 8, 10, 10, 10, 10, 12, 20]",5.0,6.0,0.0,5.0,0.0,6.0,3.0,19.0
454,South Coast J,South Coast J,FSCJ02-2_59.jpg,72341609,0,0.0,0.00,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,22.0,0.0


# A look into the results

## The sum of the clustering
What is the sum of the methods

In [30]:
df_comparison_yes_no[df_comparison_yes_no["data.yes"] >= 5]

,site_name,subject_group,image_name,subject_id,count_total,median_count,mean_count,mode_min_count,mode_max_count,mode_count,...,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points,data.no,data.yes
10,Gardner Islet,Gardner Islet,EGI04-1_141.jpg,72333244,3,2.0,3.54,2.0,2.0,[2],...,46.0,"[2, 2, 2, 2, 2, 2, 2, 2, 4, 6, 6, 6, 8]",2.0,3.0,0.0,5.0,0.0,0.0,0.0,23.0
14,Gardner Islet,Gardner Islet,EGI08-2_78.jpg,72333835,1,2.0,2.00,2.0,2.0,[2],...,12.0,"[2, 2, 2, 2, 2, 2]",2.0,1.0,0.0,5.0,0.0,6.0,10.0,12.0
38,Punta Suarez,Punta Suarez,EPS01_28.jpg,72335422,1,2.0,2.00,2.0,2.0,[2],...,12.0,"[2, 2, 2, 2, 2, 2]",1.0,1.0,0.0,5.0,0.0,6.0,14.0,8.0
39,Punta Suarez,Punta Suarez,EPS01_41.jpg,72335435,1,4.0,3.33,4.0,4.0,[4],...,30.0,"[2, 2, 2, 4, 4, 4, 4, 4, 4]",2.0,2.0,0.0,5.0,0.0,0.0,5.0,17.0
41,Punta Suarez,Punta Suarez,EPS02_208.jpg,72335541,2,4.0,4.46,4.0,4.0,[4],...,58.0,"[2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 14]",2.0,2.0,0.0,5.0,0.0,8.0,1.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,South Coast J,South Coast J,FSCJ02-2_220.jpg,72341559,1,2.0,2.00,2.0,2.0,[2],...,22.0,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]",3.0,1.0,0.0,5.0,0.0,16.0,0.0,21.0
450,South Coast J,South Coast J,FSCJ02-2_233.jpg,72341570,1,2.0,3.00,2.0,2.0,[2],...,18.0,"[2, 2, 2, 2, 2, 8]",1.0,1.0,0.0,5.0,0.0,12.0,9.0,12.0
451,South Coast J,South Coast J,FSCJ02-2_243.jpg,72341576,3,2.0,3.20,2.0,2.0,[2],...,32.0,"[2, 2, 2, 2, 2, 2, 4, 4, 4, 8]",2.0,2.0,0.0,5.0,0.0,4.0,6.0,15.0
453,South Coast J,South Coast J,FSCJ02-2_269.jpg,72341593,5,10.0,9.45,10.0,10.0,[10],...,104.0,"[4, 6, 6, 8, 8, 10, 10, 10, 10, 12, 20]",5.0,6.0,0.0,5.0,0.0,6.0,3.0,19.0


In [34]:
# Take only the subjects which have more or equal 5 yes votes.
df_method_sums = df_comparison_yes_no[df_comparison_yes_no["data.yes"] >= 5][["count_total", "median_count", "mean_count", "mode_min_count", "mode_max_count", "mode_count_avg", "mode_min_count", "mode_max_count", "dbscan_count_sil", "HDBSCAN_count"]].sum()
df_method_sums.to_csv(config["method_sums"])
df_method_sums

count_total          589.000000
median_count         988.000000
mean_count          1008.890000
mode_min_count       956.000000
mode_max_count      1004.000000
mode_count_avg       979.333333
mode_min_count       956.000000
mode_max_count      1004.000000
dbscan_count_sil     524.000000
HDBSCAN_count        589.000000
dtype: float64

## Compare the numbers
The counts are only for images which were in the dataset after filtering.

### Sum of all the Methods

In [32]:
print(f"{config['method_sums'].name}")
df_method_sums = pd.read_csv(config["method_sums"])
df_method_sums

Iguanas 2nd launch_method_sums.csv


,Unnamed: 0,0
0,median_count,988.000000
1,mean_count,1008.890000
2,mode_min_count,956.000000
3,mode_max_count,1004.000000
4,mode_count_avg,979.333333
5,mode_min_count,956.000000
6,mode_max_count,1004.000000
7,dbscan_count_sil,524.000000
8,HDBSCAN_count,589.000000


### Comparison per Image Level

In [33]:
print(f"load {config['comparison_dataset']}")
pd.read_csv(config["comparison_dataset"])

load /Users/christian/data/zooniverse/2024_04_12_analysis/Iguanas 2nd launch/Iguanas 2nd launch_method_comparison.csv


,Unnamed: 0,image_name,subject_id,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,0,EGI01-1-1_19.jpg,72332775,2.0,2.00,2,2,[2],2.0,1,2,[2],0.0,0.0,0.0,0.0,0,0.0
1,1,EGI01-1-1_54.jpg,72332788,2.0,2.00,2,2,[2],2.0,1,2,[2],0.0,0.0,0.0,0.0,0,0.0
2,2,EGI01-1-2_110.jpg,72332799,24.0,24.00,24,24,[24],24.0,1,24,[24],3.0,1.0,0.0,5.0,0,6.0
3,3,EGI01-1-2_121.jpg,72332802,54.0,54.00,54,54,[54],54.0,1,54,[54],6.0,5.0,0.0,5.0,0,0.0
4,4,EGI01-1-2_122.jpg,72332803,6.0,5.67,6,6,[6],6.0,6,34,"[2, 4, 6, 6, 6, 10]",2.0,3.0,0.0,5.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3507,3507,ESCG02-2_59.jpg,72373345,2.0,2.00,2,2,[2],2.0,2,4,"[2, 2]",0.0,0.0,0.0,0.0,0,0.0
3508,3508,ESCG02-2_62.jpg,72373347,2.0,2.00,2,2,[2],2.0,4,8,"[2, 2, 2, 2]",0.0,1.0,0.0,5.0,0,2.0
3509,3509,ESCG02-2_63.jpg,72373348,2.0,2.00,2,2,[2],2.0,4,8,"[2, 2, 2, 2]",0.0,1.0,0.0,5.0,0,2.0
3510,3510,ESCG02-2_64.jpg,72373350,14.0,12.80,10,14,"[10, 14]",12.0,5,64,"[10, 10, 14, 14, 16]",2.0,6.0,0.0,5.0,0,4.0
